In [1]:
import sys
import random
import copy
import numpy as np
import scipy as sp
import pandas as pd
import rebound
import reboundx
from astropy.coordinates import SkyCoord
from astropy import units as u
from astropy.coordinates import (CartesianRepresentation,  CartesianDifferential)
import astropy.coordinates as coord
from astropy.coordinates import Galactic

# IMPORTANT: Set whether to migrate Uranus and Neptune or not; Set initial chemical (CO/CO2) distributions

In [2]:
migration = "UNMigration" #"NoMigration" "UNMigration"
chemdist = "no_overlap" #"binary" "linear" "no_overlap" No Overlap is for OC sims. Rest are for SD.
galactictides = "off" #"on" "off"
stellarperturb = "off" #"on" "off"

# Set parameters

In [3]:
#Set locations for comets
def comets_loc(inner_lim, outer_lim, n_comets):
    comets = randomstate.uniform(low = inner_lim, high = outer_lim , size = n_comets)
    return comets

In [4]:
#Set specific random seed for each cluster node sim, used to replicate for later
seedinput = 1 #int(sys.argv[1])
randomstate = np.random.RandomState(seedinput)
#np.random.seed(seed=seedinput)
filenum = seedinput

n_comets = 50
#Set timesteps (tau = planetary migration timescale recalculation, enc = stellar encounter samples, applies after planetary migration is finished)
tau_timestep_num = 100
enc_timestep_num = 449901 #449901 #always one more than (tmax - t_ce) / desired interval between steps
#Set interval at which rebound simulation snapshot is taken
sim_archive_interval = 5e5 #5e5

#Integrator used by simulation
sim_integrator = "mercurius" #"ias15" "whfast "mercurius"

#Set distance (in terms of Hill Radii) at which integrator switches to IAS15 to handle close encounters (for Mercurius)
RcriticalMerc = 6.

#Set distance (in AU) at which objects are removed from the sim
object_exit_distance = 40000 #206266 is 1 pc #1e99 for no removals

if chemdist == "binary" or chemdist == "linear":
    a_comets = comets_loc(5.,45.,n_comets)
elif chemdist == "no_overlap":
    a_comets = comets_loc(29.,34.,n_comets)
e_comets = comets_loc(0.15,0.15,n_comets)

sim_name = "5-40 AU Initial Distribution"

#tmax is the total simulation time
#T_ce is the time at which the planetary bodies stop moving (but the comets will move)
tmax = 4.5e9 #1.2e6 #4.5e9
T_ce = 1e6

#Initial estimate of mean time between stellar encounters, ignore if no perturbation is allowed
t_m = 3.4e4

#Time intervals in which comets are cloned, ignore if not cloning comets/no particle removal
t_clone = [0.1e9,0.5e9,1e9,2e9,3.5e9] #[0.09e6,0.49e6,0.99e6,1.99e6,3.49e6] #[0.099e9,0.499e9,0.999e9,1.999e9,3.499e9]
#t_clone_f = [0.101e9,0.501e9,1.001e9,2.001e9,3.501e9] #[0.11e6,0.51e6,1.01e6,2.01e6,3.51e6] #[0.101e9,0.501e9,1.001e9,2.001e9,3.501e9]

#Set the semi-major axis and eccentricity timescales for Uranus and Neptune only
if migration == "UNMigration":
    a_i_list = [5.2044, 9.5826, 17.5, 27.5]
    e_i_list = [0.0489, 0.0565, 0.2, 0.3]
elif migration == "NoMigration":
    a_i_list = [5.2044, 9.5826, 19.2184, 30.1104]
    e_i_list = [0.0489, 0.0565, 0.0463, 0.00946]

a_f_list = [5.2044, 9.5826, 19.2184, 30.1104]
e_f_list = [0.0489, 0.0565, 0.0463, 0.00946]

#Sun's influence envelope (dependant on distance from galaxy centre) is 1 pc at 8 kpc from centre of Milky Way
r_max = 1.

#Define spectral type and associated parameters
type_names = ["B9-B0","A0","A5","F0","F5","G0","G5","K0","K5","M0","M5-M9","White Dwarf","Giants"]
stellar_masses = [9.0,2.7,1.9,1.5,1.3,1.1,0.94,0.79,0.61,0.42,0.20,0.90,4.0]
mass_densities = [0.06,0.27,0.44,1.42,0.64,1.52,2.34,2.68,5.26,8.72,41.55,3.00,0.43]
v_apexs = [18.6,17.1,13.7,17.1,17.1,26.4,23.9,19.8,25.0,17.3,23.3,38.3,21.0]
v_dispersions = [8.5,11.4,13.7,16.8,20.9,21.6,22.6,19.7,25.1,24.7,24.1,36.6,23.7]

stellar_data = pd.DataFrame({ 'type_name' : type_names,
                              'stellar_mass' : stellar_masses,
                              'mass_density' : mass_densities,
                              'v_apex' : v_apexs,
                              'v_dispersion' : v_dispersions,
                            })
#stellar_data

## Setup for filenames later on

In [5]:
if sim_integrator == "ias15":
    integrator_filelabel = "_IAS15_"
elif sim_integrator == "mercurius":
    integrator_filelabel = "_Rcrit"+str(RcriticalMerc)+"_"
    
if galactictides == "on":
    tides_filelabel = "_Tides"
else:
    tides_filelabel = ""
    
if stellarperturb == "on":
    perturb_filelabel = "_Perturb"
else:
    perturb_filelabel = ""

# Main Function Declarations

## Chemical Composition Tracers

In [6]:
def chemratio(a):
    if chemdist == "binary":
        if a > 20.:
            CO2 = 1.
        else:
            CO2 = 0
        if a > 30.:
            CO = 1.
        else:
            CO = 0
        return (CO,CO2)
    
    elif chemdist == "no_overlap":
        if 29. <= a < 31.5:
            CO2 = 1.
            CO = 0
        elif 31.5 <= a <= 34.: 
            CO = 1.
            CO2 = 0
        else:
            pass
        return (CO,CO2)
    
    elif chemdist == "linear":
        return 0.1005*a - 0.995

## Tau

In [7]:
#Define Tau values
def Tau(x_i,x_f,T_ce,t):
    Tau = (T_ce/(2.*np.log(x_i/x_f)))/(1.-t/(T_ce))
    return abs(float(Tau))

## Stellar Perturbation Derivation

In [8]:
#Define relevant stellar perturbation equations (distances in pc)

def entry_point_funct(r_max,theta,phi):   
    Rx = r_max*(np.sin(theta)*np.cos(phi))
    Ry = r_max*(np.sin(theta)*np.sin(phi))
    Rz = r_max*(np.cos(phi))
    return (Rx, Ry, Rz)    

def peculiar_v_funct(v_dispersions):
    eta_list = []
    for i in range(3):
        eta = 2.*(np.sqrt(3./5.))*(np.sum(randomstate.random_sample()+randomstate.random_sample()+randomstate.random_sample()
                                          +randomstate.random_sample()+randomstate.random_sample()))-np.sqrt(3.*5.)
        eta_list.append(eta)
    v_pe = v_dispersions*(np.sqrt((eta_list[0]**2 + eta_list[1]**2 + eta_list[2]**2)/3.))
    return v_pe

def spectral_type_funct(df,v_bracket_list):
    S_k_list, S_k_1_list = [], []
    S_k, S_k_1, S = 0., 0., 0.
    for (mass_density,v_bracket) in zip(df["mass_density"],v_bracket_list):
        S += mass_density*v_bracket
    for i in range(1,len(df["mass_density"])):
        S_k += df["mass_density"][i]*v_bracket_list[i]
        S_k_1 += df["mass_density"][i-1]*v_bracket_list[i-1]
        S_k_list.append(S_k)
        S_k_1_list.append(S_k_1)
    while True:
        rand = randomstate.uniform(0.,1.)
        for (S_k, S_k_1) in zip(S_k_list, S_k_1_list):
            if S_k_1/S <= rand <= S_k/S:
                #Add 1 to index as sum starts at 1
                return S_k_list.index(S_k)+1
        

def v_mag_funct(v_pe,v_apex):
    v_mag = np.sqrt(v_pe**2 + v_apex**2 - 2.*v_pe*v_apex*randomstate.uniform(-1.,1.))
    return v_mag

def v_funct(v_mag,psi,theta,phi):
    v3 = -v_mag*np.sqrt(randomstate.random_sample())
    v1 = np.sqrt(v_mag**2 - v3**2)*np.cos(psi)
    v2 = np.sqrt(v_mag**2 - v3**2)*np.sin(psi)
    Vx = -v1*np.sin(phi) + v2*np.cos(theta)*np.cos(phi) + v3*np.sin(theta)*np.cos(phi)
    Vy = v1*np.cos(phi) + v2*np.cos(theta)*np.sin(phi) + v3*np.sin(theta)*np.sin(phi)
    Vz = -v2*np.sin(theta) + v3*np.cos(theta)
    return (Vx, Vy, Vz)


## Drawing sample for stellar perturbation

In [9]:
#Draws a single sample for stellar perturbations
def starsamplefunct(stellar_data,r_max):
    #Draw random theta and phi for entry point and velocity angles
    theta = 2.*np.pi*randomstate.random_sample()
    phi = 2.*np.pi*randomstate.random_sample()
    #print("theta =",theta,", phi =",phi)

    #Calculate peculiar velocity of each spectral type given dispersion
    v_pe_list = []
    for sigma in stellar_data["v_dispersion"]:
        v_pe = peculiar_v_funct(sigma)
        v_pe_list.append(v_pe)

    #print(v_pe_list)
    stellar_data["v_pe"] = v_pe_list

    #Calculate velocity(?) of each spectral type
    v_bracket_list = []
    for (v_apex,v_pe) in zip(stellar_data["v_apex"],v_pe_list):
        v_bracket = np.sqrt(v_pe**2 + v_apex**2)
        v_bracket_list.append(v_bracket)

    #print(v_bracket_list)
    stellar_data["v_bracket"] = v_bracket_list

    #print(stellar_data)

    #Determine spectral type given distributions of stellar data and v_bracket
    spectral_type = spectral_type_funct(stellar_data,v_bracket_list)
    print("Spectral type =",stellar_data["type_name"][spectral_type])

    #print("v_apex =",stellar_data["v_apex"][spectral_type])

    #For that spectral type, find the magnitude of the velocity
    v_magnitude = v_mag_funct(v_pe_list[spectral_type],stellar_data["v_apex"][spectral_type])
    #print("v_magnitude =",v_magnitude)

    #While Vi/V0i < some random number, reiterate v_magnitude caculation
    v_0i = 5.*stellar_data["v_dispersion"][spectral_type]
    #print(v_magnitude/v_0i_list[spectral_type])
    while v_magnitude/v_0i >= randomstate.random_sample():
        v_pe_recalc = peculiar_v_funct(stellar_data["v_dispersion"][spectral_type])
        v_magnitude = v_mag_funct(v_pe_recalc,stellar_data["v_apex"][spectral_type])

    #print("v_magnitude_final =",v_magnitude)

    #Find entry velocity (in Euclidean space, km/s) of a passing star
    psi = 2.*np.pi*randomstate.random_sample()
    Vx,Vy,Vz = v_funct(v_magnitude,psi,theta,phi)
    print("Vx =",Vx,"Vy =",Vy,"Vz =",Vz)
    
    #Find entry position (in Euclidean space, pc) of passing star
    Rx,Ry,Rz = entry_point_funct(r_max,theta,phi)
    print("Rx =",Rx,"Ry =",Ry,"Rz =",Rz)
    
    return (v_bracket_list,spectral_type,Vx,Vy,Vz,Rx,Ry,Rz)

## Probability of perturbation actually occurring

In [10]:
 #Find out if star is added to sim
def perturb_probability_funct(sim,galactictides,stellar_data,spectral_type,r_max,v_bracket_list,Rx,Ry,Rz,Vx,Vy,Vz,t_m,time):
    #Convert mass density (in units of 10^-3 M_sun per pc^3) to number density per cubed parsec, 
    #calculate encounter frequency (in 1/seconds)
    stellar_data["number_density"] = (1e-3)*stellar_data["mass_density"]/stellar_data["stellar_mass"]
    #print(stellar_data["number_density"])
    freq = (np.pi/3.086e13)*v_bracket_list[0]*stellar_data["number_density"][0]*(r_max)**2
    for i in range(1,13):
        freq += (np.pi/3.086e13)*v_bracket_list[i]*stellar_data["number_density"][i]*(r_max)**2
    #Convert seconds to years to input into sim
    freq = freq*3.154e7
    #print("Frequency =",freq)
    probability = np.exp(-freq*(time-t_m))
    #print("Perturbation Probability =", probability)
    encounter_YN = randomstate.random_sample()
    if probability >= encounter_YN:
        print("Encounter")
        print("t = ",sim.t)
        #Add star to sim, convert pc to AU, convert km/s to AU/yr
        t_m = sim.t + 1./freq
        #Test with massless stars
        sim.add(m=0, #stellar_data["stellar_mass"][spectral_type],
                x=206265.*Rx, y=206265.*Ry, z=206265.*Rz, 
                vx=0.210805*Vx, vy=0.210805*Vy, vz=0.210805*Vz)
        if galactictides == "on":
            c = SkyCoord(sim.particles[-1].x, sim.particles[-1].y, sim.particles[-1].z, \
                         unit='AU', frame='barycentrictrueecliptic',representation_type='cartesian')
            c_fk5 = SkyCoord(c.transform_to(frame='galactic', merge_attributes=True) , \
                             representation_type='cartesian')


            c1 = coord.BarycentricTrueEcliptic(x = sim.particles[-1].x*u.AU, \
                                           y = sim.particles[-1].y*u.AU, \
                                           z = sim.particles[-1].z*u.AU, \
                                           v_x=sim.particles[-1].vx*u.AU/u.year, \
                                           v_y=sim.particles[-1].vy*u.AU/u.year,\
                                           v_z=sim.particles[-1].vz*u.AU/u.year,\
                                           differential_type='cartesian',\
                                           representation_type='cartesian')

            gc_frame = coord.Galactic(differential_type='cartesian',representation_type='cartesian' )
            gc2 = c1.transform_to(gc_frame)


            sim.particles[-1].x = -np.asscalar(c_fk5.u/(u.AU))
            sim.particles[-1].y =  np.asscalar( c_fk5.v/(u.AU))
            sim.particles[-1].z = -np.asscalar(c_fk5.w/(u.AU))

            sim.particles[-1].vx = -0.210805*np.asscalar(gc2.velocity.d_x/(u.km/u.s))
            sim.particles[-1].vy =  0.210805*np.asscalar(gc2.velocity.d_y/(u.km/u.s))
            sim.particles[-1].vz = -0.210805*np.asscalar(gc2.velocity.d_z/(u.km/u.s)) 
        else:
            pass
    #Ensures that encounter occurs if above consistently fails
    elif probability < 1e-2:
        print("Encounter")
        print("t = ",sim.t)
        t_m = sim.t + 1./freq
        sim.add(m=0, #stellar_data["stellar_mass"][spectral_type], 
                x=206265.*Rx, y=206265.*Ry, z=206265.*Rz, 
                vx=0.210805*Vx, vy=0.210805*Vy, vz=0.210805*Vz)
        if galactictides == "on":
            c = SkyCoord(sim.particles[-1].x, sim.particles[-1].y, sim.particles[-1].z, \
                         unit='AU', frame='barycentrictrueecliptic',representation_type='cartesian')
            c_fk5 = SkyCoord(c.transform_to(frame='galactic', merge_attributes=True) , \
                             representation_type='cartesian')


            c1 = coord.BarycentricTrueEcliptic(x = sim.particles[-1].x*u.AU, \
                                           y = sim.particles[-1].y*u.AU, \
                                           z = sim.particles[-1].z*u.AU, \
                                           v_x=sim.particles[-1].vx*u.AU/u.year, \
                                           v_y=sim.particles[-1].vy*u.AU/u.year,\
                                           v_z=sim.particles[-1].vz*u.AU/u.year,\
                                           differential_type='cartesian',\
                                           representation_type='cartesian')

            gc_frame = coord.Galactic(differential_type='cartesian',representation_type='cartesian' )
            gc2 = c1.transform_to(gc_frame)


            sim.particles[-1].x = -np.asscalar(c_fk5.u/(u.AU))
            sim.particles[-1].y =  np.asscalar( c_fk5.v/(u.AU))
            sim.particles[-1].z = -np.asscalar(c_fk5.w/(u.AU))

            sim.particles[-1].vx = -0.210805*np.asscalar(gc2.velocity.d_x/(u.km/u.s))
            sim.particles[-1].vy =  0.210805*np.asscalar(gc2.velocity.d_y/(u.km/u.s))
            sim.particles[-1].vz = -0.210805*np.asscalar(gc2.velocity.d_z/(u.km/u.s)) 
        else:
            pass
    else:
        pass
        #print("No Encounter")
    #print("")
    return sim, t_m

## Integration code

In [11]:
if chemdist == "binary" or chemdist == "no_overlap":
    def do_calc(sim,t,CO_amount,CO2_amount,non_stellar_objects):
        while True:
            try:
                sim.integrate(t)
                break
            #Remove all stars that exceed r_max
            except rebound.Escape as error:
                #print(error)
                for j in range(sim.N):
                    p = sim.particles[j]
                    d2 = p.x*p.x + p.y*p.y + p.z*p.z
                    if d2 > sim.exit_max_distance**2:
                        index = j # cache index rather than remove here since our loop would go beyond end of particles array
                sim.remove(index=index)
                #Note that CO/CO2 lists begin with the first comet (object 5 and up), ignore if object doesnt have chemratio (ex. star)
                try:
                    del CO_amount[index-4]
                    del CO2_amount[index-4]
                    del non_stellar_objects[index]
                    #print("Object number:",index)
                    #print("Comet number:",index-4)
                    
                    CO_file = open("CO_"+migration+"_"+str(filenum)+integrator_filelabel+chemdist+"chem"+tides_filelabel+
                                   perturb_filelabel+".txt","w+")
                    CO_file.write(str(",".join(str(amount) for amount in CO_amount)))
                    CO_file.close()

                    CO2_file = open("CO2_"+migration+"_"+str(filenum)+integrator_filelabel+chemdist+"chem"+tides_filelabel+
                               perturb_filelabel+".txt","w+")
                    CO2_file.write(str(",".join(str(amount) for amount in CO2_amount)))
                    CO2_file.close()

                    objects_file = open("NonStellarObjects_"+migration+"_"+str(filenum)+integrator_filelabel+chemdist+"chem"
                                        +tides_filelabel+perturb_filelabel+".txt","w+")
                    objects_file.write(str(",".join(non_stellar_objects)))
                    objects_file.close()

                except Exception:
                    pass #print("Star removed") #pass
        #orbits = sim.calculate_orbits()
        #orbits_list.append(orbits)    
        return sim, CO_amount, CO2_amount, non_stellar_objects

elif chemdist == "linear":
    def do_calc(sim,t,CO_ratio,non_stellar_objects):
        while True:
            try:
                sim.integrate(t)
                break
            #Remove all non_stellar_objects that exceed r_max
            except rebound.Escape as error:
                #print(error)
                for j in range(sim.N):
                    p = sim.particles[j]
                    d2 = p.x*p.x + p.y*p.y + p.z*p.z
                    if d2 > sim.exit_max_distance**2:
                        index = j # cache index rather than remove here since our loop would go beyond end of particles 
                                  # array
                sim.remove(index=index)
                #Note that CO/CO2 lists begin with the first comet (object 5 and up), ignore if object doesnt have 
                #chemratio (ex. star)
                try:
                    del CO_ratio[index-4]
                    del non_stellar_objects[index]
                    #print("Object number:",index)
                    #print("Comet number:",index-4)
                    #print("After removal:",CO_ratio)
                    
                    CO_ratio_file = open("CO_ratio_"+migration+"_"+str(filenum)+integrator_filelabel+chemdist+"chem"+
                                         tides_filelabel+perturb_filelabel+".txt","w+")
                    CO_ratio_file.write(str(",".join(str(ratio) for ratio in CO_ratio)))
                    CO_ratio_file.close()

                    objects_file = open("NonStellarObjects_"+migration+"_"+str(filenum)+integrator_filelabel+chemdist+
                                        "chem"+tides_filelabel+perturb_filelabel+".txt","w+")
                    objects_file.write(str(",".join(non_stellar_objects)))
                    objects_file.close()

                except Exception:
                    pass #print("Star removed") #pass
        #orbits = sim.calculate_orbits()
        #orbits_list.append(orbits)    
        return sim, CO_ratio, non_stellar_objects

## Comet cloning function

In [12]:
#Comet cloning function
if chemdist == "binary" or chemdist == "no_overlap":
    def comets_clone(sim,CO_amount,CO2_amount,non_stellar_objects):
        for i in range(n_comets-(len(non_stellar_objects)-4)):
            if sim.particles[i+4].m == 0.:
                #Clone comet with slight variation in mean anomaly
                #print("Comet cloned")
                sim.add(a=sim.particles[i+4].a, e=sim.particles[i+4].e, inc=sim.particles[i+4].inc, 
                        Omega=sim.particles[i+4].Omega,omega=sim.particles[i+4].omega, 
                        M=(sim.particles[i+4].M)+(randomstate.random_sample()*2*np.pi*1e-6))
                non_stellar_objects.append(str(i))
                CO_amount.append(CO_amount[i+4])
                CO2_amount.append(CO2_amount[i+4])

                CO_file = open("CO_"+migration+"_"+str(filenum)+integrator_filelabel+chemdist+"chem"+tides_filelabel+
                           perturb_filelabel+".txt","w+")
                CO_file.write(str(",".join(str(amount) for amount in CO_amount)))
                CO_file.close()

                CO2_file = open("CO2_"+migration+"_"+str(filenum)+integrator_filelabel+chemdist+"chem"+tides_filelabel+
                               perturb_filelabel+".txt","w+")
                CO2_file.write(str(",".join(str(amount) for amount in CO2_amount)))
                CO2_file.close()

                objects_file = open("NonStellarObjects_"+migration+"_"+str(filenum)+integrator_filelabel+chemdist+"chem"+tides_filelabel+
                               perturb_filelabel+".txt","w+")
                objects_file.write(str(",".join(non_stellar_objects)))
                objects_file.close()

        return sim, CO_amount, CO2_amount, non_stellar_objects
        
elif chemdist == "linear":
    def comets_clone(sim,CO_ratio,non_stellar_objects):
        for i in range(n_comets-(len(non_stellar_objects)-4)):
            if sim.particles[i+4].m == 0.:
                #Clone comet with slight variation in mean anomaly
                #print("Comet cloned")
                sim.add(a=sim.particles[i+4].a, e=sim.particles[i+4].e, inc=sim.particles[i+4].inc, 
                        Omega=sim.particles[i+4].Omega,omega=sim.particles[i+4].omega, 
                        M=(sim.particles[i+4].M)+(randomstate.random_sample()*2*np.pi*1e-6))
                non_stellar_objects.append(str(i))
                CO_ratio.append(CO_ratio[i+4])

                CO_ratio_file = open("CO_ratio_"+migration+"_"+str(filenum)+integrator_filelabel+chemdist+"chem"+
                                     tides_filelabel+perturb_filelabel+".txt","w+")
                CO_ratio_file.write(str(",".join(str(ratio) for ratio in CO_ratio)))
                CO_ratio_file.close()

                objects_file = open("NonStellarObjects_"+migration+"_"+str(filenum)+integrator_filelabel+chemdist+"chem"
                                    +tides_filelabel+perturb_filelabel+".txt","w+")
                objects_file.write(str(",".join(non_stellar_objects)))
                objects_file.close()

        return sim, CO_ratio, non_stellar_objects

# Galactic tides frame conversions

In [13]:
def tides_barytogalactic(sim,n_comets):
    # The galactic tides equations were derived in the galactic frame of reference, so we need to change the frame.
    # We do this for both positions and velocities for all particles.
    # we are transforming from "barycentrictrueecliptic" to "galactic" using "cartesian" coordinates.
    for particleind in range(0, len(sim.particles)):
        c = SkyCoord(sim.particles[particleind].x, sim.particles[particleind].y, sim.particles[particleind].z, \
                     unit='AU', frame='barycentrictrueecliptic',representation_type='cartesian')
        c_fk5 = SkyCoord(c.transform_to(frame='galactic', merge_attributes=True) , \
                         representation_type='cartesian')


        c1 = coord.BarycentricTrueEcliptic(x = sim.particles[particleind].x*u.AU, \
                                       y = sim.particles[particleind].y*u.AU, \
                                       z = sim.particles[particleind].z*u.AU, \
                                       v_x=sim.particles[particleind].vx*u.AU/u.year, \
                                       v_y=sim.particles[particleind].vy*u.AU/u.year,\
                                       v_z=sim.particles[particleind].vz*u.AU/u.year,\
                                       differential_type='cartesian',\
                                       representation_type='cartesian')

        gc_frame = coord.Galactic(differential_type='cartesian',representation_type='cartesian' )
        gc2 = c1.transform_to(gc_frame)


        sim.particles[particleind].x = -np.asscalar(c_fk5.u/(u.AU))
        sim.particles[particleind].y =  np.asscalar( c_fk5.v/(u.AU))
        sim.particles[particleind].z = -np.asscalar(c_fk5.w/(u.AU))

        sim.particles[particleind].vx = -0.210805*np.asscalar(gc2.velocity.d_x/(u.km/u.s))
        sim.particles[particleind].vy =  0.210805*np.asscalar(gc2.velocity.d_y/(u.km/u.s))
        sim.particles[particleind].vz = -0.210805*np.asscalar(gc2.velocity.d_z/(u.km/u.s))    
    return sim


def tides_galactictobary(sim,n_comets):
    # Transform back into the barycentric ecliptic frame
    for particleind in range (0, len(sim.particles)):
        c = SkyCoord(-sim.particles[particleind].x, sim.particles[particleind].y, -sim.particles[particleind].z, \
                     unit='AU', frame='galactic',representation_type='cartesian')
        c_fk5 = SkyCoord(c.transform_to(frame='barycentrictrueecliptic', merge_attributes=True) , \
                         representation_type='cartesian')


        c = Galactic(u= -sim.particles[particleind].x*u.AU,\
                     v=  sim.particles[particleind].y*u.AU,\
                     w= -sim.particles[particleind].z*u.AU, \
                     U= -sim.particles[particleind].vx*u.AU/u.year, \
                     V=  sim.particles[particleind].vy*u.AU/u.year,\
                     W= -sim.particles[particleind].vz*u.AU/u.year,\
                     representation_type='cartesian',differential_type='cartesian')

        gc_frame = coord.BarycentricTrueEcliptic(differential_type='cartesian', representation_type='cartesian')

        gc2 = c.transform_to(gc_frame)


        sim.particles[particleind].x = np.asscalar(c_fk5.x/(u.AU))
        sim.particles[particleind].y = np.asscalar(c_fk5.y/(u.AU))
        sim.particles[particleind].z = np.asscalar(c_fk5.z/(u.AU))

        sim.particles[particleind].vx = 0.210805*np.asscalar(gc2.velocity.d_x/(u.km/u.s))
        sim.particles[particleind].vy = 0.210805*np.asscalar(gc2.velocity.d_y/(u.km/u.s))
        sim.particles[particleind].vz = 0.210805*np.asscalar(gc2.velocity.d_z/(u.km/u.s))
    return sim

## Creating the simulation instance

In [14]:
def simulation_create(migration,tmax,T_ce,t_m,sim_archive_interval,t_clone,n_comets,tau_timestep_num,enc_timestep_num,a_i_list,
                      a_f_list,e_i_list,e_f_list,a_comets,RcriticalMerc,object_exit_distance,chemdist,stellar_data,r_max,
                      sim_name,filenum):
    #Call simulations stuffs
    sim = rebound.Simulation()
    sim.units = ('yr', 'AU', 'Msun')

    #Retrieve planet data from file
    try:
        sim = rebound.Simulation.from_file("./solarsystem_"+migration+".bin")
    except:
        sim.add('sun')
        sim.add('jupiter')
        sim.add('saturn')
        if migration == "UNMigration":
            sim.add(m=4.3645e-5,a=a_i_list[2], e=e_i_list[2], inc=0., Omega=0, omega=0, f=0 )
            sim.add(m=5.1483e-5,a=a_i_list[3], e=e_i_list[3], inc=0., Omega=0, omega=0, f=0 )
        elif migration == "NoMigration":
            sim.add('uranus')
            sim.add('neptune')
        sim.save("./solarsystem_"+migration+".bin")
    
    non_stellar_objects = ["Jupiter","Saturn","Uranus","Neptune"]
    
    sim.integrator = sim_integrator
    sim.collision = "mercurius"
    sim.collision_resolve = "merge"
    sim.collision_resolve_keep_sorted = 1
    #sim.boundary = "open"

    if ((sim.integrator == "whfast") | (sim.integrator == "mercurius")):
        sim.dt =  sim.particles[1].P/40.

    sim.ri_mercurius.rcrit = RcriticalMerc

    #Call reboundx for additional forces
    rebx = reboundx.Extras(sim)
    #Add in force that determines planet migration 
    #rebx.add("modify_orbits_forces")
    mod_effect = rebx.add("modify_orbits_direct")
    mod_effect.params["p"] = 0.
    if galactictides == "on":
        # load galactic tides module from reboundx
        #print("Load galactic tides module")
        galtides = rebx.add("galactic_tides")
        # Different galactic parameters (distance from centre, density, rot speed..)
        # These have been transformed to  yr-AU-Msun  system instead of kpc etc...
        galtides.params['galac_delta'] = 0.
        galtides.params['distance_galac_cent'] = 1.65e9  #AU, 8kpc  
        galtides.params['galac_ang_speed'] = 26. * 1.02e-9
        galtides.params['galac_density'] = 0.1/(8.77e+15)
        galtides.params['grav_constant'] = 4.*(3.14**2.)
    else:
        pass
    
    #NOTE: Changing tau_a (as it increases with respect to time) DOES NOT alter the integration time, instead,
    #tau_a merely alters acceleration (which then determines position)
    #The amount of integration steps are independent to tau_a recalculations, 
    #and merely uses acceleration to calculate orbit parameters
    #i.e. integration time will be same regardless of tau_a
    #Computing time only increases because python now has to calculate through the following functions
   
    CO_amount, CO2_amount, CO_ratio = [], [], []

    for i in xrange(0,n_comets): 
        rand = randomstate.random_sample()*2*np.pi
        sim.add(a=a_comets[i], e=e_comets[i], inc=0., Omega=0, omega=rand, f=rand)
        non_stellar_objects.append(str(i))
        if chemdist == "binary" or chemdist == "no_overlap":
            sim.particles[-1].params["CO"],sim.particles[-1].params["CO2"] = chemratio(a_comets[i])
            CO_amount.append(chemratio(a_comets[i])[0])
            CO2_amount.append(chemratio(a_comets[i])[1])
        elif chemdist == "linear":
            sim.particles[-1].params["CO/CO2"] = chemratio(a_comets[i])
            CO_ratio.append(chemratio(a_comets[i]))
    
    #print("CO :",CO_amount)
    #print("CO2 :",CO2_amount)
     
    #Save chem ratios as text file for later reading
    if chemdist == "binary" or chemdist == "no_overlap":
        CO_file = open("CO_"+migration+"_"+str(filenum)+integrator_filelabel+chemdist+"chem"+tides_filelabel+
                        perturb_filelabel+".txt","w+")
        CO_file.write(str(",".join(str(amount) for amount in CO_amount)))
        CO_file.close()

        CO2_file = open("CO2_"+migration+"_"+str(filenum)+integrator_filelabel+chemdist+"chem"+tides_filelabel+
                                   perturb_filelabel+".txt","w+")
        CO2_file.write(str(",".join(str(amount) for amount in CO2_amount)))
        CO2_file.close()

    elif chemdist == "linear":
        CO_ratio_file = open("CO_ratio_"+migration+"_"+str(filenum)+integrator_filelabel+chemdist+"chem"+tides_filelabel+
                                   perturb_filelabel+".txt","w+")
        CO_ratio_file.write(str(",".join(str(ratio) for ratio in CO_ratio)))
        CO_ratio_file.close()

    objects_file = open("NonStellarObjects_"+migration+"_"+str(filenum)+integrator_filelabel+chemdist+"chem"+
                        tides_filelabel+perturb_filelabel+".txt","w+")
    objects_file.write(str(",".join(non_stellar_objects)))
    objects_file.close()

    sim.move_to_com()
    sim.exit_max_distance = object_exit_distance
    orbits_list = []
    
    #Run galactic tides frame conversion function (barycentric to galactic)
    if galactictides == "on":
        #print("Switched to galactic frame")
        sim = tides_barytogalactic(sim,n_comets)
    else:
        pass
    
    sim.move_to_com()
    
    #Set simulationarchive to reload later in case sim is interrupted
    sim.automateSimulationArchive("sim_save_"+migration+"_"+str(filenum)+integrator_filelabel+chemdist+"chem"+
                                  tides_filelabel+perturb_filelabel+".bin",
                                  interval=sim_archive_interval,deletefile=False)
    
    #Timesteps at which tau is recalculated (note that perturbation sampling also occurs here at the same timestep)
    tau_times = np.linspace(0, T_ce, tau_timestep_num,endpoint=False)
    #Timesteps at which sampler is attempted
    enc_times = np.linspace(T_ce, tmax, enc_timestep_num)
    
    #ONLY USE IF NOT USING simulationarchive
    #t_list = tau_times.tolist()
    #for enc_time in enc_times:
    #    t_list.append(enc_time)

    #Simulate planet migration
    for time in tau_times:
        if migration == "UNMigration":
            sim.particles[3].params["tau_a"] = Tau(a_i_list[2],a_f_list[2],T_ce,sim.t)
            sim.particles[4].params["tau_a"] = Tau(a_i_list[3],a_f_list[3],T_ce,sim.t)
            sim.particles[3].params["tau_e"] = -Tau(e_i_list[2],e_f_list[2],T_ce,sim.t)
            sim.particles[4].params["tau_e"] = -Tau(e_i_list[3],e_f_list[3],T_ce,sim.t)
        elif migration == "NoMigration":
            sim.particles[3].params["tau_a"] = np.inf
            sim.particles[4].params["tau_a"] = np.inf
            sim.particles[3].params["tau_e"] = -np.inf
            sim.particles[4].params["tau_e"] = -np.inf
        else:
            pass
            
        #Do stellar perturbations as well for each timestep
        if stellarperturb == "on":
            if (time >= t_m):
                v_bracket_list,spectral_type,Vx,Vy,Vz,Rx,Ry,Rz = starsamplefunct(stellar_data,r_max)
                sim, t_m = perturb_probability_funct(sim,galactictides,stellar_data,spectral_type,r_max,
                                                     v_bracket_list,Rx,Ry,Rz,Vx,Vy,Vz,t_m,time)
            else:
                pass
        else:
            pass
        
        #With SimulationArchive saving structure
        if chemdist == "binary" or chemdist == "no_overlap":
            sim, CO_amount, CO2_amount, non_stellar_objects = do_calc(sim,time,CO_amount,CO2_amount,non_stellar_objects)
        elif chemdist == "linear":
            sim, CO_ratio, non_stellar_objects = do_calc(sim,time,CO_ratio,non_stellar_objects)
        
        #With no saving structure
        #if chemdist == "binary" or chemdist == "no_overlap":
        #    sim,orbits_list,CO_amount,CO2_amount,non_stellar_objects = do_calc(sim,time,orbits_list,CO_amount,CO2_amount,non_stellar_objects)
        #elif chemdist == "linear":
        #    sim,orbits_list,CO_ratio,non_stellar_objects = do_calc(sim,time,orbits_list,CO_ratio,non_stellar_objects)
            
            
    #Simulate adding of stars into relevant envelope of influence
    #print("t > T_ce")
    for t in enc_times:
        print(sim.t)
        #print(str(sim_name)+" Working on t > T_ce")
        sim.particles[3].params["tau_a"] = np.inf #after time reaches T_ce, tau_a becomes infinity (time for planets to move additional distance is infinite, i.e. plaents don't move)
        sim.particles[4].params["tau_a"] = np.inf
        sim.particles[3].params["tau_e"] = -np.inf
        sim.particles[4].params["tau_e"] = -np.inf
        
        #Clone comets to maintain n_comets with slight variation in mean anomaly
        try:
            for i in t_clone:
                if (sim.t == i) and (n_comets-(len(non_stellar_objects)-4) > 0):
                    #print(sim.t)
                    if chemdist == "binary" or chemdist == "no_overlap":
                        sim,CO_amount,CO2_amount,non_stellar_objects = comets_clone(sim,CO_amount,CO2_amount,
                                                                                    non_stellar_objects)
                    elif chemdist == "linear":
                        sim,CO_ratio,non_stellar_objects = comets_clone(sim,CO_ratio,non_stellar_objects)
                    t_clone.remove(i)
                    #print(t_clone)
                else:
                    pass
        except:
            pass
        
        #Do stellar perturbations0
        if stellarperturb == "on":
            if (t >= t_m):
                v_bracket_list,spectral_type,Vx,Vy,Vz,Rx,Ry,Rz = starsamplefunct(stellar_data,r_max)
                sim,t_m = perturb_probability_funct(sim,galactictides,stellar_data,spectral_type,r_max,v_bracket_list,
                                                    Rx,Ry,Rz,Vx,Vy,Vz,t_m,t)
            else:
                pass
        else:
            pass
        
        #With simulationarchive saving structure
        if chemdist == "binary" or chemdist == "no_overlap":
            sim, CO_amount, CO2_amount, non_stellar_objects = do_calc(sim,t,CO_amount,CO2_amount,non_stellar_objects)
        elif chemdist == "linear":
            sim, CO_ratio, non_stellar_objects = do_calc(sim,t,CO_ratio,non_stellar_objects)
        else:
            pass
        
        #With no saving structure
        #if chemdist == "binary" or chemdist == "no_overlap":
        #    sim,orbits_list,CO_amount,CO2_amount,non_stellar_objects = do_calc(sim,t,orbits_list,CO_amount,CO2_amount,non_stellar_objects)
        #elif chemdist == "linear":
        #    sim,orbits_list,CO_ratio,non_stellar_objects = do_calc(sim,t,orbits_list,CO_ratio,non_stellar_objects)
        #else:
        #    pass
    
    #Run galactic tides frame conversion function (galactic to barycentric)
    if galactictides == "on":
        #print("Switched to barycentric frame")
        sim = tides_galactictobary(sim,n_comets)
    else:
        pass
    
    #Delete sim
    #del sim
    
    #Get orbital parameters at each rebound snapshot
    sa = rebound.SimulationArchive("sim_save_"+migration+"_"+str(filenum)+integrator_filelabel+chemdist+"chem"+
                              tides_filelabel+perturb_filelabel+".bin")
    
    t_list = np.linspace(0.,tmax,(tmax/sim_archive_interval)+1)
    orbits_list = []
    for i, sim in enumerate(sa):
        orbits_list.append(sim.calculate_orbits())
        
    #print("CO :",CO_amount)
    #print("CO2 :",CO2_amount)
    
    if chemdist == "binary" or chemdist == "no_overlap":
        return sim_name, orbits_list, t_list, CO_amount, CO2_amount, non_stellar_objects
    elif chemdist == "linear":
        return sim_name, orbits_list, t_list, CO_ratio, non_stellar_objects
    else:
        pass

## Loading simulation (if applicable)

In [15]:
def simulation_load(migration,tmax,T_ce,t_m,sim_archive_interval,t_clone,tau_timestep_num,enc_timestep_num,
                    RcriticalMerc,object_exit_distance,chemdist,stellar_data,r_max,sim_name,filenum):
    #load Saved Simulation archive
    sa = rebound.SimulationArchive("sim_save_"+migration+"_"+str(filenum)+integrator_filelabel+chemdist+"chem"+
                                   tides_filelabel+perturb_filelabel+".bin")
    print("sim_save_"+migration+"_"+str(filenum)+integrator_filelabel+chemdist+"chem"+
                                   tides_filelabel+perturb_filelabel+".bin")
    print("Number of snapshots: %d" % len(sa))
    print("Time of first and last snapshot: %.1f, %.1f" % (sa.tmin, sa.tmax))
    sim_saved = sa[-1]  
    
    sim_saved.integrator = sim_integrator
    sim_saved.collision = "mercurius"
    sim_saved.collision_resolve = "merge"
    sim_saved.collision_resolve_keep_sorted = 1
    #sim_saved.boundary = "open"

    if ((sim_saved.integrator == "whfast") | (sim_saved.integrator == "mercurius")):
        sim_saved.dt =  sim_saved.particles[1].P/40.  

    sim_saved.ri_mercurius.rcrit = RcriticalMerc

    rebx = reboundx.Extras(sim_saved)
    mod_effect = rebx.add("modify_orbits_direct")
    mod_effect.params["p"] = 0.  
    if galactictides == "on":
        # load galactic tides module from reboundx
        galtides = rebx.add("galactic_tides")
        # Different galactic parameters (distance from centre, density, rot speed..)
        # These have been transformed to  yr-AU-Msun  system instead of kpc etc...
        galtides.params['galac_delta'] = 0.
        galtides.params['distance_galac_cent'] = 1.65e9  #AU, 8kpc  
        galtides.params['galac_ang_speed'] = 26. * 1.02e-9
        galtides.params['galac_density'] = 0.1/(8.77e+15)
        galtides.params['grav_constant'] = 4.*(3.14**2.)
    else:
        pass
    
    sim_saved.move_to_com()
    sim_saved.exit_max_distance = object_exit_distance
  
    if chemdist == "binary" or chemdist == "no_overlap":
        CO_file = open("CO_"+migration+"_"+str(filenum)+integrator_filelabel+chemdist+"chem"+tides_filelabel+
                        perturb_filelabel+".txt","r")
        CO_amount_txt = CO_file.read()
        CO_amount_str = CO_amount_txt.split(",")
        CO_amount = [float(i) for i in CO_amount_str]

        CO2_file = open("CO2_"+migration+"_"+str(filenum)+integrator_filelabel+chemdist+"chem"+tides_filelabel+
                        perturb_filelabel+".txt","r")
        CO2_amount_txt = CO2_file.read()
        CO2_amount_str = CO2_amount_txt.split(",")
        CO2_amount = [float(i) for i in CO2_amount_str]

        objects_file = open("NonStellarObjects_"+migration+"_"+str(filenum)+integrator_filelabel+chemdist+"chem"+
                            tides_filelabel+perturb_filelabel+".txt","r")
        non_stellar_objects = objects_file.read()
        non_stellar_objects = non_stellar_objects.split(",")

    elif chemdist == "linear":
        CO_ratio_file = open("CO_ratio_"+migration+"_"+str(filenum)+integrator_filelabel+chemdist+"chem"+tides_filelabel+
                        perturb_filelabel+".txt","r")
        CO_ratio_txt = CO_ratio_file.read()
        CO_ratio_str = CO_ratio_txt.split(",")
        CO_ratio = [float(i) for i in CO_ratio_str]

        objects_file = open("NonStellarObjects_"+migration+"_"+str(filenum)+integrator_filelabel+chemdist+"chem"+
                            tides_filelabel+perturb_filelabel+".txt","r")
        non_stellar_objects = objects_file.read()
        non_stellar_objects = non_stellar_objects.split(",")
    else:
        pass
    
    t_load = sim_saved.t
    
    '''i = 0
    orbits_saved = sim_saved.calculate_orbits()
    for orbit_saved in orbits_saved:
        if i < 4:
            print(i,", ",orbit_saved.a,", ",orbit_saved.e,sep="")
        elif 4 <= i < len(sim_saved.particles):
            if chemdist == "binary" or chemdist == "no_overlap":
                print(i,", ",orbit_saved.a,", ",orbit_saved.e,", ",CO_amount[i-4],", ",CO2_amount[i-4],sep="")
            elif chemdist == "linear":
                print(i,", ",orbit_saved.a,", ",orbit_saved.e,", ",CO_ratio[i-4],sep="")
        else:
            break
        i += 1
    print("")'''
    
    #Actual time at which integrator starts up
    enc_times = np.linspace(t_load, tmax, enc_timestep_num)

    #Set simulationarchive to reload later in case sim is interrupted
    sim_saved.automateSimulationArchive("sim_save_"+migration+"_"+str(filenum)+integrator_filelabel+chemdist+"chem"+
                                  tides_filelabel+perturb_filelabel+".bin",
                                  interval=sim_archive_interval,deletefile=False)
    
    #Simulate adding of stars into relevant envelope of influence
    #print("t > T_ce")
    for t in enc_times:
        #print(str(sim_name)+" Working on t > T_ce")
        sim_saved.particles[3].params["tau_a"] = np.inf #after time reaches T_ce, tau_a becomes infinity (time for planets to move additional distance is infinite, i.e. plaents don't move)
        sim_saved.particles[4].params["tau_a"] = np.inf
        sim_saved.particles[3].params["tau_e"] = -np.inf
        sim_saved.particles[4].params["tau_e"] = -np.inf
        
        #Clone comets to maintain n_comets with slight variation in mean anomaly
        try:
            for i in t_clone:
                if (sim.t == i) and (n_comets-(len(non_stellar_objects)-4) > 0):
                    #print(sim.t)
                    if chemdist == "binary" or chemdist == "no_overlap":
                        sim,CO_amount,CO2_amount,non_stellar_objects = comets_clone(sim,CO_amount,CO2_amount,
                                                                                    non_stellar_objects)
                    elif chemdist == "linear":
                        sim,CO_ratio,non_stellar_objects = comets_clone(sim,CO_ratio,non_stellar_objects)
                    t_clone.remove(i)
                    #print(t_clone)
                else:
                    pass
        except:
            pass
        
        #Do stellar perturbations as well for each timestep
        if stellarperturb == "on":
            if (t >= t_m):
                v_bracket_list,spectral_type,Vx,Vy,Vz,Rx,Ry,Rz = starsamplefunct(stellar_data,r_max)
                sim_saved,t_m = perturb_probability_funct(sim_saved,galactictides,stellar_data,spectral_type,r_max,
                                                          v_bracket_list,Rx,Ry,Rz,Vx,Vy,Vz,t_m,t)
            else:
                pass
        else:
            pass
        
        if chemdist == "binary" or chemdist == "no_overlap":
            sim_saved, CO_amount, CO2_amount, non_stellar_objects = do_calc(sim_saved,t,CO_amount,CO2_amount,
                                                                                                non_stellar_objects)
        elif chemdist == "linear":
            sim_saved, CO_ratio, non_stellar_objects = do_calc(sim_saved,t,CO_ratio,non_stellar_objects)
        else:
            pass
            
    #Run galactic tides frame conversion function (galactic to barycentric)
    if galactictides == "on":
        sim_saved = tides_galactictobary(sim_saved,n_comets)
    else:
        pass
            
    #Delete sim
    #del sim_saved

    #Get orbital parameters at each rebound snapshot
    sa_1 = rebound.SimulationArchive("sim_save_"+migration+"_"+str(filenum)+integrator_filelabel+chemdist+"chem"+
                              tides_filelabel+perturb_filelabel+".bin")
    
    t_list = np.linspace(0.,tmax,(tmax/sim_archive_interval)+1)
    orbits_list = []
    for i, sim_saved in enumerate(sa_1):
        orbits_list.append(sim_saved.calculate_orbits())
        
    print(t_list.tolist())
    print("")
    for orbits in orbits_list:
        i = 0
        for orbit in orbits:
            if i < 4:
                print(str(i)+", "+str(orbit.a)+", "+str(orbit.e))
            elif 4 <= i < len(non_stellar_objects):
                if chemdist == "binary" or chemdist == "no_overlap":
                    print(str(i)+", "+str(orbit.a)+", "+str(orbit.e)+", "+str(CO_amount[i-4])+", "
                                            +str(CO2_amount[i-4]))
                elif chemdist == "linear":
                    print(str(i)+", "+str(orbit.a)+", "+str(orbit.e)+", "+str(CO_ratio[i-4]))
            else:
                break
            i += 1
        print("")

# Call Simulations

In [16]:
try:
    simulation_load(migration,tmax,T_ce,t_m,sim_archive_interval,t_clone,tau_timestep_num,enc_timestep_num,
                    RcriticalMerc,object_exit_distance,chemdist,stellar_data,r_max,sim_name,filenum)
except ValueError:
    import time
    t0 = time.time()
    print("Load error, creating new simulation ...")
    if chemdist == "binary" or chemdist == "no_overlap":
        sim_name, orbits_list, t_list, CO_amount, CO2_amount, non_stellar_objects = simulation_create(migration,tmax,
                                                                T_ce,t_m,sim_archive_interval,t_clone,n_comets,
                                                                tau_timestep_num,enc_timestep_num,a_i_list,a_f_list,
                                                                e_i_list,e_f_list,a_comets, RcriticalMerc,
                                                                object_exit_distance,chemdist,
                                                                stellar_data,r_max,sim_name,filenum)
    elif chemdist == "linear":
        sim_name, orbits_list, t_list, CO_ratio, non_stellar_objects = simulation_create(migration,tmax,T_ce,t_m,
                                                                sim_archive_interval,t_clone,n_comets,
                                                                tau_timestep_num,enc_timestep_num,a_i_list,a_f_list,
                                                                e_i_list,e_f_list,a_comets,RcriticalMerc,
                                                                object_exit_distance,chemdist,stellar_data,r_max,sim_name,
                                                                filenum)
    else:
        pass
    print(t_list.tolist())
    print("")
    for orbits in orbits_list:
        i = 0
        for orbit in orbits:
            #print(i)
            if i < 4:
                print(str(i)+", "+str(orbit.a)+", "+str(orbit.e))
            elif 4 <= i < len(non_stellar_objects):
                if chemdist == "binary" or chemdist == "no_overlap":
                    print(str(i)+", "+str(orbit.a)+", "+str(orbit.e)+", "+str(CO_amount[i-4])+", "
                                            +str(CO2_amount[i-4]))
                elif chemdist == "linear":
                    print(str(i)+", "+str(orbit.a)+", "+str(orbit.e)+", "+str(CO_ratio[i-4]))
            else:
                break
            i += 1
        print("")
    t1 = time.time()
    total = t1 - t0
    #print(total)

sim_save_UNMigration_1_Rcrit6.0_no_overlapchem.bin
Number of snapshots: 863
Time of first and last snapshot: 0.0, 219500000.0
0, 5.2031614639731645, 0.029485299702234433
1, 9.541771806040671, 0.061226837797622814
2, 19.13094780667853, 0.0751121740810213
3, 30.565702750827153, 0.016237095911066046
4, 59.18646940109625, 0.5588346674491582, 0.0, 1.0
5, 106.38087683970375, 0.6963798926242569, 0.0, 1.0
6, 46.418317095937105, 0.19305923097108552, 0.0, 1.0
7, 10394.49669434032, 0.9976172260981059, 0.0, 1.0
8, 2708.9097588257005, 0.9883295119053477, 1.0, 0.0
9, 45.34616291792955, 0.769114506206205, 0.0, 1.0
10, 168.3773303870671, 0.7961455834788885, 1.0, 0.0
11, 278.6484055284056, 0.9061017464580182, 1.0, 0.0
12, 558.846081520278, 0.9422561242134257, 0.0, 1.0
13, 25.773634435258526, 0.5364864948174803, 0.0, 1.0
14, 93.57743057926525, 0.6987236138752531, 1.0, 0.0
15, 112.59760092625234, 0.757195930145483, 0.0, 1.0
16, 296.585171213786, 0.8919696875345738, 0.0, 1.0
17, 74.65374544269442, 0.57478

/home/Michael/anaconda3/lib/python3.6/site-packages/rebound/simulationarchive.py:127: RuntimeWarning: You have to reset function pointers after creating a reb_simulation struct with a binary file.
  warnings.warn(message, RuntimeWarning)
/home/Michael/anaconda3/lib/python3.6/site-packages/rebound/simulation.py:505: RuntimeWarning: File in use for SimulationArchive already exists. Snapshots will be appended.
  warnings.warn(msg[1:], RuntimeWarning)


KeyboardInterrupt: 

In [ ]:
'''#Plot orbit distances wrt time

#Pick five random comets to plot
e_orbits_list = []
for orbits in orbits_list:
    e_orbit_singletime = []
    for i in range(len(orbits)):
        if i < 5:
            e_orbit_singletime.append(orbits[i].e)
    e_orbits_list.append(e_orbit_singletime)
e_orbits_list = np.transpose(e_orbits_list)
plot_e_time(e_orbits_list,t_list,"test")'''